In [1]:
!git clone https://github.com/tiendatnguyen2266/Do-An-2
%cd Do-An-2
!pip -q install -r setup.txt

Cloning into 'Do-An-2'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 0), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 77.12 KiB | 38.56 MiB/s, done.
/content/Do-An-2


In [2]:
!pip -q install pyngrok

In [3]:
import torch
from diffusers import StableDiffusionPipeline

# Định nghĩa tham số
rand_seed = torch.manual_seed(42)
NUM_INFERENCE_STEPS = 50
GUIDANCE_SCALE = 0.75
HEIGHT = 512
WIDTH = 512

# Danh sách model
model_list = ["nota-ai/bk-sdm-small",
              "CompVis/stable-diffusion-v1-4",
              "runwayml/stable-diffusion-v1-5",
              "prompthero/openjourney",
              "stabilityai/stable-diffusion-xl-base-1.0",
              "stabilityai/stable-diffusion-3.5-large",
              "dreamlike-art/dreamlike-photoreal-2.0"
              ]


def create_pipeline(model_name = model_list[5]):
    # Nếu máy có GPU CUDA
    if torch.cuda.is_available():
        print("Using GPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype = torch.float16,
            use_safetensors = True
        ).to("cuda")
    elif torch.backends.mps.is_available():
        print("Using MPS")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to("mps")
    else:
        print("Using CPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float32,
            use_safetensors=True
        )
    return pipeline

def text2img(prompt, pipeline):
    images = pipeline(
        prompt,
        guidance_scale = GUIDANCE_SCALE,
        num_inference_steps = NUM_INFERENCE_STEPS,
        generator = rand_seed,
        num_images_per_request = 1,
        height = HEIGHT,
        width = WIDTH
    ).images

    return images[0]


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from flask import Flask, request, render_template
from text2img_model import create_pipeline, text2img
from pyngrok import ngrok, conf

from google.colab import userdata


# Khởi tạo Flask app
app =  Flask(__name__)

# Định nghĩa tham số
IMAGE_PATH  = "static/output.jpg"
conf.get_default().auth_token = userdata.get('ngrok_token')

public_url = ngrok.connect(8888).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


# Khởi tạo pipeline
pipeline = create_pipeline()

@app.route("/", methods = ['GET', 'POST'])
def index():
    # Kiểm tra xem có phải người dùng view trang web không
    if request.method == "GET":
        # Trả về giao diện trang web
        return render_template("index.html")
    else:
        # Xử lý việc người dùng submit prompt-> sinh ảnh -> trả về
        user_input = request.form["prompt"]

        print("Start gen....")
        im = text2img(user_input, pipeline)
        print("Finish gen....")
        im.save(IMAGE_PATH)

        return render_template("index.html", image_url = IMAGE_PATH)

if __name__ =='__main__':
    app.run(debug=False, host='0.0.0.0', port=8888, use_reloader=False)


 * ngrok tunnel "https://382a-34-126-132-59.ngrok-free.app" -> "http://127.0.0.1:5000/"
Using GPU


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8888
 * Running on http://172.28.0.12:8888
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Nov/2024 06:03:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2024 06:03:32] "GET /favicon.ico HTTP/1.1" 404 -


Start gen....


...

Start gen....


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Nov/2024 06:04:01] "POST / HTTP/1.1" 200 -


Finish gen....


INFO:werkzeug:127.0.0.1 - - [24/Nov/2024 06:04:02] "GET /static/output.jpg HTTP/1.1" 200 -
